In [1]:
import pandas as pd
pd.set_option("display.max_row", None)

In [2]:
%load_ext autoreload
%autoreload 2

In [34]:
deal_data = {
    "name":"Sample with Interest Swap"
    ,"dates":{"cutoff":"2021-06-01"
              ,"closing":"2021-07-15"
              ,"firstPay":"2021-08-26"
              ,"payFreq":["DayOfMonth",20]
              ,"poolFreq":"MonthEnd"
              ,"stated":"2030-01-01"}
    ,"pool":{'assets':[["Mortgage"
                        ,{"originBalance":2200,"originRate":["fix",0.045],"originTerm":30
                          ,"freq":"Monthly","type":"Level","originDate":"2021-02-01"}
                          ,{"currentBalance":2200
                          ,"currentRate":0.08
                          ,"remainTerm":30
                          ,"status":"current"}]]
            }
    ,"accounts":{"acc01":{"balance":0}}
    ,"bonds":{"A1":{"balance":1000
                 ,"rate":0.07
                 ,"originBalance":1000
                 ,"originRate":0.07
                 ,"startDate":"2020-01-03"
                 ,"rateType":{"Fixed":0.08}
                 ,"bondType":{"Sequential":None}}
             ,"B":{"balance":1000
                     ,"rate":0.0
                     ,"originBalance":1000
                     ,"originRate":0.07
                     ,"startDate":"2020-01-03"
                     ,"rateType":{"Fixed":0.00}
                     ,"bondType":{"Equity":None}}}
    ,"fees":{"trusteeFee":{"type":{"fixFee":30}}}
    ,"collect":[["CollectedInterest","acc01"]
              ,["CollectedPrincipal","acc01"]
              ,["CollectedPrepayment","acc01"]
              ,["CollectedRecoveries","acc01"]]
    ,"waterfall":{"Amortizing":[
         ["payFee","acc01",['trusteeFee']]         
         ,["accrueAndPayInt","acc01",["A1"]]
         ,["payPrin","acc01",["A1"]]
         ,["payPrin","acc01",["B"]]
         ,["payPrinResidual","acc01",["B"]]
     ]}
    ,"status":("PreClosing","Amortizing")
}

In [4]:
from absbox import API,mkDeal,EnginePath,SPV
localAPI = API(EnginePath.DEV,check=False)

trigger = {
    "AfterCollect":
      {"poolDef":
        {"condition":[("cumPoolDefaultedBalance",),">",20]
        ,"effects":("newStatus","Defaulted")
        ,"status":False
        ,"curable":False}
      }
}

deal = mkDeal(deal_data|{"triggers":trigger})

r = localAPI.run(deal
                 ,poolAssump = ("Pool"
                                ,("Mortgage"
                                 ,{"CDR":0.05} ,None, None, None)
                                 ,None
                                 ,None)
                 ,runAssump = [("inspect"
                                ,("MonthEnd",("cumPoolDefaultedBalance",)))
                              ,("call",{"afterDate":"2022-09-01"})
                              #,("fireTrigger",[("2021-10-01","AfterCollect","poolDef")])
                              ]
                 ,read=True)

#r['pool']['flow']

Connecting engine server -> https://absbox.org/api/dev

✅Connected, local lib:0.26.4, server:0.28.8

NameError: name 'deal_data' is not defined

In [5]:
test01 = SPV(
     "未满足摊还目标的加速清偿事件"
     ,{"封包日":"2021-03-01","起息日":"2021-05-01","首次兑付日":"2021-06-26"
      ,"法定到期日":"2060-12-01","收款频率":"月末","付款频率":["每月",26]}     
     ,{'清单':[["按揭贷款"
         ,{"放款金额":120,"放款利率":["固定",0.0485],"初始期限":30
           ,"频率":"每月","类型":"等额本金","放款日":"2020-06-01"}
           ,{"当前余额":120
           ,"当前利率":0.08
           ,"剩余期限":10
           ,"状态":"正常"}]],
      }
     ,(("账户01",{"余额":0}),)
     ,(("A1",{"当前余额":100
              ,"当前利率":0.07
              ,"初始余额":100
              ,"初始利率":0.07
              ,"起息日":"2020-01-03"
              ,"利率":{"固定":0.08}
              ,"债券类型":{"固定摊还":[["2021-06-26",80]
                                     ,["2021-07-26",70]
                                     ,["2021-09-26",50]
                                     ,["2021-11-26",0]]}
              })
       ,("B",{"当前余额":40
              ,"当前利率":0.08
              ,"初始余额":100
              ,"初始利率":0.07
              ,"起息日":"2020-01-03"
              ,"利率":{"固定":0.08}
              ,"债券类型":{"过手摊还":None}
              }))
     ,tuple()
     ,{("兑付日","摊销"):[
          ["计提支付利息","账户01",["A1"]]
          ,["计提支付利息","账户01",["B"]]
          ,["支付本金","账户01",["A1"]]
          ,["支付本金","账户01",["B"]]
       ]
       ,("兑付日","加速清偿"):[
          ["计提支付利息","账户01",["A1"]]
          ,["支付剩余本金","账户01",["A1"]]
         ]      
      ,"回款后":[]
      ,"清仓回购":[]}
     ,(["利息回款","账户01"]
       ,["本金回款","账户01"]
       ,["早偿回款","账户01"]
       ,["回收回款","账户01"])
     ,None
     ,None
     ,None
     ,{"分配后":{"目标余额测试":
                  {"条件":[("债券低于目标余额","A1") ,">",0]
                  ,"效果":("新状态","加速清偿")
                  ,"状态":False
                  ,"重置":False}
                  }
               }
    ,("设计","摊销")
 )


In [6]:
r = localAPI.run(test01
                 ,poolAssump = None
                 ,runAssump = []
                 ,read=True)

Warning Message from server:
Bond B is not paid off
Account 账户01 has cash to be distributed